# Convert EBM to IRM

This notebook takes the three-layer energy balance model tunings from Donald Cummins and converts them to a three-layer impulse response function.

It will then save these into a CSV file.

In [1]:
import os

import numpy as np
import pandas as pd
import scipy.linalg
from tqdm import tqdm

from fair21.constants.general import (
    EARTH_RADIUS,
    SECONDS_PER_YEAR,
    DOUBLING_TIME_1PCT
)

In [2]:
df = pd.read_csv(
    os.path.join("..", "data", "calibration", "4xCO2_cummins.csv")
)

In [3]:
models = df['model'].unique()
models

array(['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-CM-1-1-MR', 'BCC-CSM2-MR',
       'BCC-ESM1', 'CAMS-CSM1-0', 'CESM2-FV2', 'CESM2-WACCM',
       'CESM2-WACCM-FV2', 'CMCC-CM2-SR5', 'CNRM-CM6-1', 'CNRM-CM6-1-HR',
       'CNRM-ESM2-1', 'CanESM5', 'E3SM-1-0', 'FGOALS-g3', 'GISS-E2-1-G',
       'GISS-E2-1-H', 'GISS-E2-2-G', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM',
       'MIROC-ES2L', 'MIROC6', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR',
       'MRI-ESM2-0', 'NorCPM1', 'SAM0-UNICON'], dtype=object)

In [4]:
params = {}
for model in models:
    params[model] = {}
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        params[model][run] = {}
        params[model][run]['gamma_autocorrelation'] = df.loc[condition, 'gamma'].values[0]
        params[model][run]['ocean_heat_capacity'] = df.loc[condition, 'C1':'C3'].values.squeeze()
        params[model][run]['ocean_heat_transfer'] = df.loc[condition, 'kappa1':'kappa3'].values.squeeze()
        params[model][run]['deep_ocean_efficacy'] = df.loc[condition, 'epsilon'].values[0]
        params[model][run]['sigma_eta'] = df.loc[condition, 'sigma_eta'].values[0]
        params[model][run]['sigma_xi'] = df.loc[condition, 'sigma_xi'].values[0]
        params[model][run]['forcing_4co2'] = df.loc[condition, 'F_4xCO2'].values[0]

In [5]:
params

{'ACCESS-CM2': {'r1i1p1f1': {'gamma_autocorrelation': 1.88578590071159,
   'ocean_heat_capacity': array([ 3.67420401, 10.0509273 , 79.00113427]),
   'ocean_heat_transfer': array([0.70452914, 3.78694388, 0.69714927]),
   'deep_ocean_efficacy': 1.43982936529305,
   'sigma_eta': 0.457682866431186,
   'sigma_xi': 0.562736202801604,
   'forcing_4co2': 7.57713207902355}},
 'ACCESS-ESM1-5': {'r1i1p1f1': {'gamma_autocorrelation': 2.536269864266,
   'ocean_heat_capacity': array([ 3.61230801, 10.36738403, 83.22149704]),
   'ocean_heat_transfer': array([0.66715386, 2.73896097, 0.92627042]),
   'deep_ocean_efficacy': 1.51117641602166,
   'sigma_eta': 0.628051953981854,
   'sigma_xi': 0.545553625801671,
   'forcing_4co2': 6.27462871581454}},
 'AWI-CM-1-1-MR': {'r1i1p1f1': {'gamma_autocorrelation': 4.3311591350037,
   'ocean_heat_capacity': array([ 4.22019861,  9.60696442, 47.65990971]),
   'ocean_heat_transfer': array([1.18366473, 1.99569026, 0.71614617]),
   'deep_ocean_efficacy': 1.39857899287456

In [6]:
# TODO: move to a constants module

forcing_2co2_4co2_ratio=0.476304  # TODO: un-hardcode this and calculate directly from Meinshausen or Etminan relations.

In [7]:
class EnergyBalanceModel:
    
    # Energy balance basics
    ocean_heat_capacity = np.array([5, 20, 100])
    ocean_heat_transfer = np.array([1, 2, 1])
    deep_ocean_efficacy = 1

    # Tuning
    forcing_4co2 = 8
    
    # Stochastic parameters (see Cummins et al. 2020)
    stochastic_run = False
    sigma_eta = 0.5
    sigma_xi = 0.5
    gamma_autocorrelation = 2

    # Output - not yet relevant
#    start = 1850.0
#    end = 2101.0
#    timestep = 0.2
#    outtime = None,
#    forcing = np.zeros(2)
    
    def __init__(self, **kwargs):
        self.ocean_heat_capacity = kwargs.get('ocean_heat_capacity', self.ocean_heat_capacity)
        self.ocean_heat_transfer = kwargs.get('ocean_heat_transfer', self.ocean_heat_transfer)
        self.deep_ocean_efficacy = kwargs.get('deep_ocean_efficacy', self.deep_ocean_efficacy)
        self.forcing_4co2 = kwargs.get('forcing_4co2', self.forcing_4co2)
        self.stochastic_run = kwargs.get('stochastic_run', self.stochastic_run)
        self.sigma_eta = kwargs.get('sigma_eta', self.sigma_eta)
        self.sigma_xi = kwargs.get('sigma_xi', self.sigma_xi)
        self.gamma_autocorrelation = kwargs.get('gamma_autocorrelation', self.gamma_autocorrelation)
    
    def _eb_matrix(self):
        # Define the matrix of differential equations
        # Cummins et al. (2020); Leach et al. (2021)
        eb_matrix = np.array(
            [
                [
                    -(self.ocean_heat_transfer[0]+self.ocean_heat_transfer[1])/self.ocean_heat_capacity[0],
                    self.ocean_heat_transfer[1]/self.ocean_heat_capacity[0], 
                    0
                ],
                [
                    self.ocean_heat_transfer[1]/self.ocean_heat_capacity[1],
                    -(self.ocean_heat_transfer[1]+self.deep_ocean_efficacy*self.ocean_heat_transfer[2])/self.ocean_heat_capacity[1],
                    self.deep_ocean_efficacy*self.ocean_heat_transfer[2]/self.ocean_heat_capacity[1]
                ],
                [
                    0, 
                    self.ocean_heat_transfer[2]/self.ocean_heat_capacity[2],
                    -self.ocean_heat_transfer[2]/self.ocean_heat_capacity[2]
                ]
            ]
        )
        return(eb_matrix)
    
        
    def impulse_response(self):
        eb_matrix = self._eb_matrix()

        # calculate the eigenvectors and eigenvalues, these are the timescales of responses
        eb_matrix_eigenvalues, eb_matrix_eigenvectors = scipy.linalg.eig(eb_matrix)
        self.timescales = -1/(np.real(eb_matrix_eigenvalues))
        self.response_coefficients = self.timescales * (eb_matrix_eigenvectors[0,:] * scipy.linalg.inv(eb_matrix_eigenvectors)[:,0]) / self.ocean_heat_capacity[0]

    def emergent_parameters(self):
        # requires impulse response step
        if not hasattr(self, 'timescales'):
            self.impulse_response()
        self.ecs = self.forcing_4co2 * forcing_2co2_4co2_ratio * np.sum(self.response_coefficients)
        self.tcr = self.forcing_4co2 * forcing_2co2_4co2_ratio * np.sum(
            self.response_coefficients*(
                1 - self.timescales/DOUBLING_TIME_1PCT * (
                    1 - np.exp(-DOUBLING_TIME_1PCT/self.timescales)
                )
            )
        )

In [8]:
ebm = EnergyBalanceModel(**params['ACCESS-CM2']['r1i1p1f1'])

In [9]:
ebm.emergent_parameters()

In [10]:
ebm.ecs

5.122596220048067

In [11]:
for model in models:
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        ebm = EnergyBalanceModel(**params[model][run])
        ebm.emergent_parameters()
        params[model][run] = ebm.__dict__

In [12]:
params

{'ACCESS-CM2': {'r1i1p1f1': {'ocean_heat_capacity': array([ 3.67420401, 10.0509273 , 79.00113427]),
   'ocean_heat_transfer': array([0.70452914, 3.78694388, 0.69714927]),
   'deep_ocean_efficacy': 1.43982936529305,
   'forcing_4co2': 7.57713207902355,
   'stochastic_run': False,
   'sigma_eta': 0.457682866431186,
   'sigma_xi': 0.562736202801604,
   'gamma_autocorrelation': 1.88578590071159,
   'timescales': array([  0.63456183,   7.75953934, 318.55119406]),
   'response_coefficients': array([0.13065828, 0.494845  , 0.79388443]),
   'ecs': 5.122596220048067,
   'tcr': 2.3458843339066955}},
 'ACCESS-ESM1-5': {'r1i1p1f1': {'ocean_heat_capacity': array([ 3.61230801, 10.36738403, 83.22149704]),
   'ocean_heat_transfer': array([0.66715386, 2.73896097, 0.92627042]),
   'deep_ocean_efficacy': 1.51117641602166,
   'forcing_4co2': 6.27462871581454,
   'stochastic_run': False,
   'sigma_eta': 0.628051953981854,
   'sigma_xi': 0.545553625801671,
   'gamma_autocorrelation': 2.536269864266,
   'tim

In [13]:
# reconstruct a data table and save
df_out = pd.DataFrame(columns=['model', 'run', 'ecs', 'tcr', 'tau1', 'tau2', 'tau3', 'q1', 'q2', 'q3'])

#values_to_add = {'A': 1, 'B': 2}
#row_to_add = pd.Series(values_to_add, name='x')

#df = df.append(row_to_add)

count = 0
for model in models:
    for run in df.loc[df['model']==model, 'run']:
        values_to_add = {
            'model': model,
            'run': run,
            'ecs': params[model][run]['ecs'],
            'tcr': params[model][run]['tcr'],
            'tau1': params[model][run]['timescales'][0],
            'tau2': params[model][run]['timescales'][1],
            'tau3': params[model][run]['timescales'][2],
            'q1': params[model][run]['response_coefficients'][0],
            'q2': params[model][run]['response_coefficients'][1],
            'q3': params[model][run]['response_coefficients'][2],
        }
        row_to_add = pd.Series(values_to_add, name=count)
        df_out = df_out.append(row_to_add)
        count = count + 1

In [14]:
df_out

,model,run,ecs,tcr,tau1,tau2,tau3,q1,q2,q3
0,ACCESS-CM2,r1i1p1f1,5.122596,2.345884,0.634562,7.759539,318.551194,0.130658,0.494845,0.793884
1,ACCESS-ESM1-5,r1i1p1f1,4.479672,1.897580,0.836798,6.439241,341.731000,0.175808,0.412565,0.910532
2,AWI-CM-1-1-MR,r1i1p1f1,3.187020,2.040492,1.091259,6.330669,165.337532,0.203445,0.307051,0.334338
3,BCC-CSM2-MR,r1i1p1f1,3.170393,1.760652,0.985343,5.805754,210.750951,0.193863,0.228822,0.408327
4,BCC-ESM1,r1i1p1f1,3.355253,1.826979,2.211158,14.723659,340.718137,0.370248,0.324109,0.527092
5,CAMS-CSM1-0,r1i1p1f1,2.234807,1.649204,0.392397,5.187294,139.272645,0.108380,0.273176,0.150579
6,CAMS-CSM1-0,r2i1p1f1,2.226454,1.645464,0.853624,5.233147,133.548573,0.121871,0.259847,0.151655
7,CESM2-FV2,r1i1p1f1,6.370097,2.148443,0.530880,4.369134,416.729061,0.086223,0.448250,1.260619
8,CESM2-WACCM,r1i1p1f1,5.282444,2.214296,0.328475,4.876315,325.545862,0.051597,0.481809,0.863547
9,CESM2-WACCM-FV2,r1i1p1f1,5.594945,2.063313,0.620994,6.503055,457.211395,0.132048,0.485243,1.158078


In [15]:
df_out.to_csv(os.path.join("..", "data", "calibration", "4xCO2_impulse_response.csv"))